In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_key import g_key

In [2]:
data = pd.read_csv("../output_data/clean_city_data.csv")

In [3]:
city_data = pd.DataFrame(data)
city_data.head()

,Unnamed: 0,City,Lat,Lng,Country,Wind,Cloud,Humidity,Max Temp.,Date
0,0,tasiilaq,65.61,-37.64,GL,2.86,96,80,33.80,1603072352
1,1,ushuaia,-54.80,-68.30,AR,10.29,75,69,39.20,1603072349
2,2,busselton,-33.65,115.33,AU,6.73,18,41,75.00,1603072348
3,3,chokurdakh,70.63,147.92,RU,5.53,95,98,19.63,1603072554
4,4,cherskiy,68.75,161.30,RU,7.11,92,93,23.68,1603072364


In [4]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
locations = city_data[["Lat","Lng"]].astype(float)
humidity = city_data["Humidity"].astype(float)

In [6]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
ideal_weather = city_data.loc[(city_data["Max Temp."] >= 70) & (city_data["Cloud"] == 0) &
                             (city_data["Wind"] < 10) & (city_data["Max Temp."] < 80)].dropna()
ideal_weather.head()

,Unnamed: 0,City,Lat,Lng,Country,Wind,Cloud,Humidity,Max Temp.,Date
18,18,cap malheureux,-19.98,57.61,MU,5.99,0,72,73.00,1603072557
41,42,nikki,9.94,3.21,BJ,5.93,0,87,73.31,1603072562
104,111,grand gaube,-20.01,57.66,MU,5.99,0,72,73.00,1603072545
130,138,potiskum,11.71,11.07,NG,9.60,0,46,78.57,1603072583
171,180,bouna,9.27,-3.00,CI,4.43,0,91,73.35,1603072592


In [8]:
# create dataframe with hotel name column
hotel_df = ideal_weather.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

In [9]:
# base url and parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}



In [10]:
# loop for city names based on lat and lng

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for {index}: {city}.")
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"The Closest Hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print(f"Result not found. Skipping...")
        
    print(f"--------------")
    time.sleep(1)
    
    
print(f"End of Search.")


Retrieving Results for 18: cap malheureux.
The Closest Hotel in cap malheureux is Paradise Cove Boutique Hotel.
--------------
Retrieving Results for 41: nikki.
The Closest Hotel in nikki is Hotel John.
--------------
Retrieving Results for 104: grand gaube.
The Closest Hotel in grand gaube is LUX Grand Gaube Resort & Villas.
--------------
Retrieving Results for 130: potiskum.
The Closest Hotel in potiskum is Tamir Hotel Ltd.
--------------
Retrieving Results for 171: bouna.
The Closest Hotel in bouna is GNIME'S HÔTEL.
--------------
Retrieving Results for 178: porto novo.
The Closest Hotel in porto novo is CTH LE PALAIS.
--------------
Retrieving Results for 229: bosaso.
The Closest Hotel in bosaso is Gacayte Hotel.
--------------
Retrieving Results for 267: tura.
The Closest Hotel in tura is Dongrama Home Stay, Tura Meghalaya.
--------------
Retrieving Results for 269: isagarh.
The Closest Hotel in isagarh is Vaibhav Furniture, Isagarh.
--------------
Retrieving Results for 276: san

In [11]:
# display dataframe
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
18,cap malheureux,MU,-19.98,57.61,Paradise Cove Boutique Hotel
41,nikki,BJ,9.94,3.21,Hotel John
104,grand gaube,MU,-20.01,57.66,LUX Grand Gaube Resort & Villas
130,potiskum,NG,11.71,11.07,Tamir Hotel Ltd
171,bouna,CI,9.27,-3.00,GNIME'S HÔTEL


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))